# Menu Controle de Despesas

In [12]:
import regex as re
import openpyxl as xl
import pandas as pd
import os


### titulos

In [13]:
titulo_empresa = """
 ██████████                                                               
▒▒███▒▒▒▒▒█                                                               
 ▒███  █ ▒  █████████████   ████████  ████████   ██████   █████   ██████  
 ▒██████   ▒▒███▒▒███▒▒███ ▒▒███▒▒███▒▒███▒▒███ ███▒▒███ ███▒▒   ▒▒▒▒▒███ 
 ▒███▒▒█    ▒███ ▒███ ▒███  ▒███ ▒███ ▒███ ▒▒▒ ▒███████ ▒▒█████   ███████ 
 ▒███ ▒   █ ▒███ ▒███ ▒███  ▒███ ▒███ ▒███     ▒███▒▒▒   ▒▒▒▒███ ███▒▒███ 
 ██████████ █████▒███ █████ ▒███████  █████    ▒▒██████  ██████ ▒▒████████
▒▒▒▒▒▒▒▒▒▒ ▒▒▒▒▒ ▒▒▒ ▒▒▒▒▒  ▒███▒▒▒  ▒▒▒▒▒      ▒▒▒▒▒▒  ▒▒▒▒▒▒   ▒▒▒▒▒▒▒▒ 
                            ▒███                                          
                            █████                                         
                           ▒▒▒▒▒      
"""

registrar_titulo = '''
______           _     _                   ______          _           
| ___ \         (_)   | |                  |  _  \        | |          
| |_/ /___  __ _ _ ___| |_ _ __ __ _ _ __  | | | |__ _  __| | ___  ___ 
|    // _ \/ _` | / __| __| '__/ _` | '__| | | | / _` |/ _` |/ _ \/ __|
| |\ \  __/ (_| | \__ \ |_| | | (_| | |    | |/ / (_| | (_| | (_) \__ \
\_| \_\___|\__, |_|___/\__|_|  \__,_|_|    |___/ \__,_|\__,_|\___/|___/
            __/ |                                                      
           |___/                                                       
'''

visualizar_titulo = '''
 _   _ _                 _ _                ______          _           
| | | (_)               | (_)               |  _  \        | |          
| | | |_ ___ _   _  __ _| |_ ______ _ _ __  | | | |__ _  __| | ___  ___ 
| | | | / __| | | |/ _` | | |_  / _` | '__| | | | / _` |/ _` |/ _ \/ __|
\ \_/ / \__ \ |_| | (_| | | |/ / (_| | |    | |/ / (_| | (_| | (_) \__ \
 \___/|_|___/\__,_|\__,_|_|_/___\__,_|_|    |___/ \__,_|\__,_|\___/|___/
                                                                        
'''

alterar_titulo = '''
  ___  _ _                       ______          _           
 / _ \| | |                      |  _  \        | |          
/ /_\ \ | |_ ___ _ __ __ _ _ __  | | | |__ _  __| | ___  ___ 
|  _  | | __/ _ \ '__/ _` | '__| | | | / _` |/ _` |/ _ \/ __|
| | | | | ||  __/ | | (_| | |    | |/ / (_| | (_| | (_) \__ \
\_| |_/_|\__\___|_|  \__,_|_|    |___/ \__,_|\__,_|\___/|___/
                                                             
'''

In [14]:
debug = False

## Menu Inicial

In [15]:
def exibir_titulo(texto):
    '''Exibe o titulo estilizado na tela'''
    print(texto)

def exibir_opcoes():
    '''Exibe todas as opcoes de input do usuario'''
    print('1. Registrar Custos')
    print('2. Visualizar Despesas')
    print('3. Alternar Dados')
    print('4. Salvar Arquivo Excel')
    print('9. Sair\n')

def finalizar_app():
    '''Exibe mensagem de finalizacao do app'''
    exibir_subtitulo('Finalizando app')

def voltar_ao_menu_principal():
    '''
    Solicita uma tecla e retorna ao menu principal
        
    Outputs:
    - Retorna ao menu principal
    '''
    input('\nDigite uma tecla para voltar ao menu ')
    main()

def opcao_invalida():
    '''Case o usuario insira um input invalido, retorne ele ao menu principal
    
    Outputs:
    - Retorna ao menu principal
    '''
    print('Input inválido! Retornando ao menu...')
    voltar_ao_menu_principal()

def exibir_subtitulo(texto):
    '''Limpa a tela e exibe o subtitulo estilizado na tela
    
    Inputs:
    - texto: str - o texto do subtitulo
    '''
    os.system('cls')
    linha = '*' * (len(texto))
    print(linha)
    print(texto)
    print(linha)
    print()

## Paginacao

In [16]:

def exibir_pagina(tabela, numero, max):
    '''
    Exibe ao usuario parte da lista de jogadoras, baseado no numero inserido (numero da pagina).
    Caso o ultimo index a ser exibido passe do valor maximo de jogadora registradas, o ultimo index é alterado para o limite.
    '''
    num_min = (numero - 1) * 10
    num_max = num_min + 10
    if num_max > max:
        num_max = max
    print(tabela[num_min:num_max])

def converter_input_numero_pagina(input_numero_pagina):
    '''
    Converte o input do usuario para um numero inteiro.
    Caso o input seja vazio ou invalido, retorna None
    '''
    try:
        input_numero_pagina = int(input_numero_pagina)
        return input_numero_pagina
    except:
        return None

def paginacao(tabela, subtitulo=str):
    '''
    Função que implementa a paginação para exibir a tabela em partes para o usuario.
    '''
    exibir_subtitulo(subtitulo)
    
    input_numero_pagina = 1
    pagina_max = len(tabela)
    numero_de_paginas = int(((pagina_max - 1)/10)+1)

    exibir_pagina(tabela, input_numero_pagina, pagina_max)
    print(f"\n-----------! Numeor de paginas: {numero_de_paginas} !----------")
    
    input_numero_pagina = converter_input_numero_pagina(input("\nInsira o numero da pagina que deseja visualizar ou pressione enter para sair: "))
    
    while isinstance(input_numero_pagina, int):
        if isinstance(input_numero_pagina, int):
            if 1 <= input_numero_pagina <= numero_de_paginas:
                exibir_subtitulo(subtitulo)
                exibir_pagina(tabela, input_numero_pagina, pagina_max)
                print(f"\n-----------! Numero de paginas: {numero_de_paginas} !----------")
                input_numero_pagina = converter_input_numero_pagina(input("\nInsira o numero da pagina que deseja visualizar ou pressione enter para sair: "))
            else:
                exibir_subtitulo(subtitulo)
                print("Numero de pagina inserido invalido.")
                print(f"\n-----------! Numero de paginas: {numero_de_paginas} !----------")
                input_numero_pagina = converter_input_numero_pagina(input("\nInsira o numero da pagina que deseja visualizar ou pressione enter para sair: "))
        else:
            print("\nRetornando ao menu principal...\n")
            voltar_ao_menu_principal()
    
def listar_jogadoras():
    '''Lista jogadoras registradas no banco de dados
    
    Inputs:
    - Dictionary de jogadoras
    
    Outputs:
    - Exibe lista de jogadoras na tela
    
    '''
    global jogadoras

    paginacao(jogadoras, "Lista de Jogadoras Cadastradas")

    voltar_ao_menu_principal()

## Registrar

In [ ]:
def exibir_opcoes_registrar():
    '''Exibe todas as opcoes de input do usuario do menu registrar dados'''
    print('1. Compras - Registro Automatico - via Arquivos XML')
    print('2. Despesas - Registro Manual')
    print('3. Servicos - Registro Manual')
    print('9. Voltar ao menu Principal\n')

def escolher_opcao_registrar():
    ''' Solicita a executa a opcao escolhida pelo usuario
    
    Outputs:
    -Executa a opcao escolhida pelo usuario
    '''
    try:
        opcao_escolhida = int(input('Escolha uma opção: '))
        # opcao_escolhida = int(opcao_escolhida)

        if opcao_escolhida == 1: 
            registro_automatico()
        elif opcao_escolhida == 2: 
            registro_despesas()
        elif opcao_escolhida == 3: 
            registro_servicos()
        elif opcao_escolhida == 9: 
            voltar_ao_menu_principal()
        else: 
            opcao_invalida()
    except Exception as e:
        if debug:
            print(f'Error: {type(e).__name__}')
            print(f"Error message: {e}")
        opcao_invalida()

def menu_registrar_dados():
    '''Menu para registrar dados de jogadoras'''
    exibir_titulo(registrar_titulo)
    exibir_opcoes_registrar()
    escolher_opcao_registrar()
    voltar_ao_menu_principal()

In [18]:
def escolher_opcao():
    ''' Solicita a executa a opcao escolhida pelo usuario
    
    Outputs:
    -Executa a opcao escolhida pelo usuario
    '''
    try:
        opcao_escolhida = int(input('Escolha uma opção: '))
        # opcao_escolhida = int(opcao_escolhida)

        if opcao_escolhida == 1: 
            menu_registrar_dados()
        elif opcao_escolhida == 2: 
            menu_visualizar_dados()
        elif opcao_escolhida == 3: 
            menu_alterar_dados()
        elif opcao_escolhida == 4: 
            salvar_arquivo_excel()
        elif opcao_escolhida == 9: 
            finalizar_app()
        else: 
            opcao_invalida()
    except Exception as e:
        if debug:
            print(f'Error: {type(e).__name__}')
            print(f"Error message: {e}")
        opcao_invalida()

def main():
    '''Funcao principal que inicial o programa'''
    os.system('cls')
    exibir_titulo(titulo_empresa)
    exibir_opcoes()
    escolher_opcao()

if __name__ == '__main__':
    main()


 ██████████                                                               
▒▒███▒▒▒▒▒█                                                               
 ▒███  █ ▒  █████████████   ████████  ████████   ██████   █████   ██████  
 ▒██████   ▒▒███▒▒███▒▒███ ▒▒███▒▒███▒▒███▒▒███ ███▒▒███ ███▒▒   ▒▒▒▒▒███ 
 ▒███▒▒█    ▒███ ▒███ ▒███  ▒███ ▒███ ▒███ ▒▒▒ ▒███████ ▒▒█████   ███████ 
 ▒███ ▒   █ ▒███ ▒███ ▒███  ▒███ ▒███ ▒███     ▒███▒▒▒   ▒▒▒▒███ ███▒▒███ 
 ██████████ █████▒███ █████ ▒███████  █████    ▒▒██████  ██████ ▒▒████████
▒▒▒▒▒▒▒▒▒▒ ▒▒▒▒▒ ▒▒▒ ▒▒▒▒▒  ▒███▒▒▒  ▒▒▒▒▒      ▒▒▒▒▒▒  ▒▒▒▒▒▒   ▒▒▒▒▒▒▒▒ 
                            ▒███                                          
                            █████                                         
                           ▒▒▒▒▒      

1. Registrar Custos
2. Visualizar Despesas
3. Alternar Dados
4. Salvar Arquivo Excel
9. Sair


______           _     _                   ______          _           
| ___ \         (_)   | |                 